In [2]:
import numpy as np

a = np.array([1,2,3,4])
print(a)

[1 2 3 4]


In [4]:
import time

a = np.random.rand(1000000)
b = np.random.rand(1000000)

tic = time.time()
c = np.dot(a, b)
toc = time.time()
print("Vectorization: "+str((toc-tic)*1000)+"ms")

Vectorization: 1.207113265991211ms


In [6]:
c=0
tic = time.time()
for i in range(1000000):
    c += a[i]*b[i]
toc = time.time()
print("For loop: "+str((toc-tic)*1000)+"ms")

For loop: 581.8850994110107ms


In [8]:
A = np.array([[5, 0, 4, 6],
            [1, 1, 5, 8],
            [1, 3, 9, 9]])
print(A)

[[5 0 4 6]
 [1 1 5 8]
 [1 3 9 9]]


In [9]:
# 对矩阵按列求和
cal = A.sum(axis=0)
print(cal)

[ 7  4 18 23]


In [10]:
# 广播
perc = 100*A/cal.reshape(1, 4)
print(perc)

[[71.42857143  0.         22.22222222 26.08695652]
 [14.28571429 25.         27.77777778 34.7826087 ]
 [14.28571429 75.         50.         39.13043478]]


## 1.1 常用函数

In [1]:
# math实现sigmoid函数
import math

def basic_sigmoid(x):
    s = 1/(1 + math.exp(-x))
    return s

basic_sigmoid(3)

0.9525741268224334

In [2]:
# numpy实现sigmoid函数
import numpy as np

def sigmoid(x):
    s = 1/(1 + np.exp(-x))
    return s

x = np.array([1, 2, 3])
sigmoid(x)

array([0.73105858, 0.88079708, 0.95257413])

In [3]:
def sigmoid_gradient(x):
    s = sigmoid(x)
    ds = s*(1-s)
    return ds

sigmoid_gradient(x)

array([0.19661193, 0.10499359, 0.04517666])

In [4]:
def softmax(x):
    x_exp = np.exp(x)
    x_sum = np.sum(x_exp, axis=1, keepdims=True)
    s = x_exp / x_sum
    return s

x2 = np.array([
    [9, 2, 5, 0, 0],
    [7, 5, 0, 0 ,0]])
softmax(x2)

array([[9.80897665e-01, 8.94462891e-04, 1.79657674e-02, 1.21052389e-04,
        1.21052389e-04],
       [8.78679856e-01, 1.18916387e-01, 8.01252314e-04, 8.01252314e-04,
        8.01252314e-04]])

In [5]:
def L1(yhat, y):
    loss = np.sum(np.abs(y - yhat))
    return loss

def L2(yhat, y):
    loss = np.dot(y-yhat, (y-yhat).T)
    return loss

yhat = np.array([.9, 0.2, 0.1, .4, .9])
y = np.array([1, 0, 0, 1, 1])
print("L1 = " + str(L1(yhat, y)))
print("L2 = " + str(L2(yhat, y)))

L1 = 1.1
L2 = 0.43


## 1.2 逻辑回归

In [6]:
import numpy as np
# 图形库
import matplotlib.pyplot as plt
# 处理H5格式的数据集
import h5py
# 科学计算库
import scipy
# 图像处理库
from PIL import Image
from scipy import ndimage
# 自定义py文件
from lr_utils import load_dataset

%matplotlib inline

In [8]:
def initialize_with_zeros(dim):
    w = np.zeros(dim, 1)
    b = 0
    return w, b

In [10]:
def propagate(w, b, X, Y):
    # training set size
    m = X.shape[1]
    # activations
    Z = np.dot(w.T, X) + b
    A = sigmoid(Z)
    # cost function
    cost = -1/m * np.sum(Y*np.log(A) + (1-Y)*np.log(1-A))
    
    # back propagation
    dZ = A - Y
    dw = 1/m * np.dot(X, dZ.T)
    db = 1/m * np.sum(dZ)
    
    grads = {
        "dw": dw,
        "db": db
    }
    cost = np.squeeze(cost)
    
    return grads, cost

In [12]:
def optimize(w, b, X, Y, iterations, learning_rate, print_cost):
    costs = []
    for i in range(iterations):
        grads, cost = propagate(w, b, X, Y)
        dw = grads["dw"]
        db = grads["db"]
        
        w = w - learning_rate * dw
        b = b - learning_rate * db
        
        if i%100 == 0:
            costs.append(cost)
            if print_cost:
                print("Cost after %i iterations: %f" %(i, cost))
    
    params = {
        "w": w,
        "b": b
    }
    grads = {
        "dw": dw,
        "db": db
    }
    return params, grads

In [ ]:
def predict(w, b, X):
    m = X.shape[1]
    Y_pred = np.zeros((1, m))
    w = w.reshape(X.shape[0], 1)
    
    A = sigmoid(np.dot(w.T, X)+b)
    for i in range(m):
        if A[0, i] <= 0.5:
            Y_pred[0, i] = 0
        else:
            Y_pred[0, i] = 1
    
    return Y_pred

In [14]:
def logistic_regression_model(X_train, Y_train, X_test, Y_test, iterations=2000, learning_rate=0.5, print_cost=False):
    # 初始化参数
    w, b = initialize_with_zeros(X_train.shape[0])
    
    # 训练，优化参数
    params, grads, costs = optimize(w, b, X_train, Y_train, iterations, learning_rate, print_cost)
    w = params["w"]
    b = params["b"]
    
    # 评估结果
    Y_pred_train = predict(w, b, X_train)
    Y_pred_test = predict(w, b, X_test)
    train_accuracy = 100 - np.mean(np.abs(Y_train - Y_pred_train))*100
    test_accuracy = 100 - np.mean(np.abs(Y_test - Y_pred_test))*100
    
    res = {
        "w": w,
        "b": b,
        "iterations": iterations,
        "learning_rate": learning_rate,
        "prediction_train": Y_pred_train,
        "train_accuracy": train_accuracy,
        "prediction_test": Y_pred_test,
        "test_accuracy": test_accuracy
    }